In [6]:
from math import sqrt
from random import randrange

preference_matrix = [[4, 3, 1, 0, 0, 1, 2, 2, 4, 2, 2, 2, 3, 3, 1],
                     [4, 0, 3, 4, 3, 0, 1, 4, 1, 2, 1, 1, 2, 2, 3],
                     [3, 2, 0, 4, 1, 0, 3, 0, 0, 4, 4, 0, 3, 0, 3],
                     [1, 0, 1, 3, 4, 1, 3, 0, 3, 2, 4, 0, 1, 2, 1],
                     [0, 4, 1, 0, 3, 0, 0, 0, 3, 1, 4, 0, 1, 1, 2],
                     [2, 1, 3, 2, 2, 3, 0, 4, 3, 0, 4, 1, 2, 4, 2],
                     [1, 4, 1, 4, 1, 0, 4, 0, 3, 3, 1, 4, 2, 0, 3],
                     [4, 1, 1, 4, 2, 3, 3, 0, 3, 4, 2, 3, 3, 3, 3],
                     [1, 1, 4, 4, 2, 3, 2, 4, 0, 1, 3, 4, 4, 0, 0],
                     [3, 2, 3, 0, 1, 4, 1, 1, 4, 4, 4, 4, 0, 1, 1],
                     [3, 4, 1, 0, 4, 0, 0, 3, 3, 4, 1, 2, 2, 3, 3],
                     [3, 3, 0, 0, 2, 1, 4, 2, 2, 2, 4, 4, 2, 3, 3],
                     [1, 3, 2, 3, 1, 2, 1, 2, 3, 2, 3, 1, 4, 1, 1],
                     [0, 1, 2, 4, 2, 0, 4, 3, 0, 4, 4, 0, 0, 4, 2],
                     [2, 1, 3, 0, 4, 0, 3, 3, 1, 2, 0, 4, 2, 0, 3]]

USERS_DICT = {i: f'U{i + 1}' for i in range(len(preference_matrix))}
PRODUCTS_DICT = {p: f'P{p + 1}' for p in range(len(preference_matrix))}
SIMILARITY_THRESHOLD = 0.7


def display_matrix(matrix, row_labels, col_labels):
    print('   ', *col_labels.values())
    for i in range(len(matrix)):
        print(list(row_labels.values())[i].ljust(3), matrix[i])
    print()


print('Дана матрица предпочтений:')
display_matrix(preference_matrix, row_labels=PRODUCTS_DICT, col_labels=USERS_DICT)


def remove_products(matrix, product_ids):
    result = []
    for i in range(len(matrix)):
        if i in product_ids:
            result.append(matrix[i])
    return result


def update_matrix(matrix, excluded_products):
    result = []
    for i in range(len(matrix)):
        row = []
        for j in range(len(matrix[i])):
            if j not in excluded_products:
                row.append(matrix[i][j])
        result.append(row)
    return result


def build_vector():
    vector = {}
    for i in range(len(preference_matrix)):
        tagA = 'P' + str(i + 1)
        for j in range(len(preference_matrix[i])):
            vector.setdefault(tagA, []).append(preference_matrix[i][j])
    return vector


def compute_similarity():
    result = {}
    for A in build_vector():
        nom = 0
        denomA = 0
        denomB = 0
        for B in build_vector():
            if A[1:] != B[1:]:
                for pair in zip(build_vector()[A], build_vector()[B]):
                    nom += (pair[0] * pair[1])
                    denomA += pair[0] ** 2
                    denomB += pair[1] ** 2
                if A + B and B + A not in result:
                    result[A + B] = (nom / (sqrt(denomA) * sqrt(denomB)))
    return result


similarity_scores = compute_similarity()

similarity_matrix = [[0] * len(preference_matrix) for i in range(len(preference_matrix))]
for key in similarity_scores:
    i, j = (int(ind) - 1 for ind in key.replace('P', ' ').strip().split())
    similarity_matrix[i][j], similarity_matrix[j][i] = similarity_scores[key], similarity_scores[key]

close_product_pairs = set()
for i in range(len(similarity_matrix)):
    for j in range(len(similarity_matrix[i])):
        if similarity_matrix[i][j] >= SIMILARITY_THRESHOLD:
            if (j, i) not in close_product_pairs and (i, j) not in close_product_pairs:
                close_product_pairs.add((i, j))

filtered_pref_matrix = remove_products(preference_matrix, [i[1] for i in close_product_pairs])
filtered_similarity_matrix = remove_products(similarity_matrix, [i[1] for i in close_product_pairs])

predicted_rating = 0
for i in range(len(filtered_pref_matrix)):
    if 0 in filtered_pref_matrix[i]:
        weighted_sum = 0
        sum_similarities = 0
        user_index = filtered_pref_matrix[i].index(0)
        target_product = PRODUCTS_DICT[list(close_product_pairs)[i][1]]
        print(f'Расчет для пользователя: [{USERS_DICT[user_index]}]. Неоцененный продукт:', [target_product])
        for j in range(len(filtered_pref_matrix)):
            if filtered_pref_matrix[j][user_index] != 0:
                rating = filtered_pref_matrix[j][user_index]
                if PRODUCTS_DICT[list(close_product_pairs)[i][1]] != PRODUCTS_DICT[list(close_product_pairs)[j][1]]:
                    try:
                        similarity = similarity_scores[
                            PRODUCTS_DICT[list(close_product_pairs)[i][1]] + PRODUCTS_DICT[
                                list(close_product_pairs)[j][1]]]
                    except KeyError:
                        similarity = similarity_scores[
                            PRODUCTS_DICT[list(close_product_pairs)[j][1]] + PRODUCTS_DICT[
                                list(close_product_pairs)[i][1]]]
                    weighted_sum += rating * similarity
                    sum_similarities += similarity

        predicted_rating = weighted_sum / sum_similarities
        print(f'Возможная оценка пользователя [{USERS_DICT[user_index]}] для продукта [{target_product}]: {predicted_rating}')
        if predicted_rating > 3:
            print(f'Рекомендовано пользователю [{USERS_DICT[user_index]}] предложить продукт [{target_product}]')
        else:
            print(f'Пользователю [{USERS_DICT[user_index]}] не рекомендовано предложить продукт [{target_product}]')
        print()

Дана матрица предпочтений:
    U1 U2 U3 U4 U5 U6 U7 U8 U9 U10 U11 U12 U13 U14 U15
P1  [4, 3, 1, 0, 0, 1, 2, 2, 4, 2, 2, 2, 3, 3, 1]
P2  [4, 0, 3, 4, 3, 0, 1, 4, 1, 2, 1, 1, 2, 2, 3]
P3  [3, 2, 0, 4, 1, 0, 3, 0, 0, 4, 4, 0, 3, 0, 3]
P4  [1, 0, 1, 3, 4, 1, 3, 0, 3, 2, 4, 0, 1, 2, 1]
P5  [0, 4, 1, 0, 3, 0, 0, 0, 3, 1, 4, 0, 1, 1, 2]
P6  [2, 1, 3, 2, 2, 3, 0, 4, 3, 0, 4, 1, 2, 4, 2]
P7  [1, 4, 1, 4, 1, 0, 4, 0, 3, 3, 1, 4, 2, 0, 3]
P8  [4, 1, 1, 4, 2, 3, 3, 0, 3, 4, 2, 3, 3, 3, 3]
P9  [1, 1, 4, 4, 2, 3, 2, 4, 0, 1, 3, 4, 4, 0, 0]
P10 [3, 2, 3, 0, 1, 4, 1, 1, 4, 4, 4, 4, 0, 1, 1]
P11 [3, 4, 1, 0, 4, 0, 0, 3, 3, 4, 1, 2, 2, 3, 3]
P12 [3, 3, 0, 0, 2, 1, 4, 2, 2, 2, 4, 4, 2, 3, 3]
P13 [1, 3, 2, 3, 1, 2, 1, 2, 3, 2, 3, 1, 4, 1, 1]
P14 [0, 1, 2, 4, 2, 0, 4, 3, 0, 4, 4, 0, 0, 4, 2]
P15 [2, 1, 3, 0, 4, 0, 3, 3, 1, 2, 0, 4, 2, 0, 3]

Расчет для пользователя: [U9]. Неоцененный продукт: ['P15']
Возможная оценка пользователя [U9] для продукта [P15]: 2.573236526329605
Пользователю [U9] не рекомендовано